# N-Gram Language Model Exercises
Inspired by Andrej Karpathy's first Makemore video. Here I'll try to reproduce what I learned from scratch and add additional features.

We'll start by making a bigram language model. We'll train it on names so it learns to produce names of its own based on what it learned from the data.

In [2]:
# PyTorch adds tensors and backpropagation.
import torch

In [3]:
# Create a list of each name in names.txt
names = open("names.txt", "r").read().splitlines()
print(f"Displaying the first three names: {names[:3]}")

Displaying the first three names: ['emma', 'olivia', 'ava']


Now that we have our data, let's split it into bigrams so the model can learn common letter patterns in names. We also want it to learn how each name starts and ends, so let's indicate the start and end of a name with a ".". Below shows an example of the bigrams in the name Emma.

In [4]:
# Break "emma" into bigrams, including "." for start and end.
print(f"Bigrams for emma: {list(zip('.emma', 'emma.'))}")

Bigrams for emma: [('.', 'e'), ('e', 'm'), ('m', 'm'), ('m', 'a'), ('a', '.')]


We want this model to take a character as input and predict which character comes next. For that, we want two tensors.

X (the input) should contain the first half of the bigrams.

y (the label) should contain the solutions, AKA the letter that comes next.

Since PyTorch tensors can't contain character elements, let's first assign each letter to an integer value.

In [5]:
import string

# For each letter + ".", add them to dict with an integer value.
stoi = {char:int_val for (int_val, char) in enumerate("." + string.ascii_lowercase)}
print(stoi)

{'.': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


<!-- Now we'll get the bigrams for all the names and split them into two tensors, X and y. -->
<!-- Now we are ready to get bigrams for all the names  and put them into tensors X and y. -->
Now we are ready to make our bigrams for the data.

In [6]:
X = [] # First letters in bigrams
y = [] # X's bigram pairs.

for name in names:
    for char1, char2 in zip("." + name, name + "."):
        # Convert chars to ints so we can later add to tensors.
        X.append(stoi[char1])
        y.append(stoi[char2])


print(f"The first 10 values in X are {X[:10]}.")
print(f"The first 10 values in y are {y[:10]}.")
# Convert list to a tensor.
X = torch.tensor(X)
y = torch.tensor(y)

The first 10 values in X are [0, 5, 13, 13, 1, 0, 15, 12, 9, 22].
The first 10 values in y are [5, 13, 13, 1, 0, 15, 12, 9, 22, 9].


<!-- Now we are ready to get started on our neural network. Before we can make input our Xs, however,  -->
<!-- Before we can input this data into our  -->
<!-- Now we are ready to get started on our neural network.  -->
<!-- For this we will use a very basic network with only a single layer of neurons. -->
Now our data is almost ready for use in our neural network. But we don't actually want our network's weights to be multiplied by the integers in X. Instead, we'll use one-hot encoding so all possible letters are treated equally: as an array of 0s and a single 1 in the index signifying our letter.

In [7]:
import torch.nn.functional as F

# One-hot encode X to turn each letter into an array of length 27 (one index for each possible letter including ".".)
Xenc = F.one_hot(X, num_classes = 27)
# The neural net needs to take in floats so it can be multiplied by float weights.
Xenc = Xenc.float()
print(Xenc[:5])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])


<!-- Now let's make our neural net. For this we'll use a very simple neural network with only a single layer of neurons.  -->
<!-- Now we're ready to make our neural net. We'll give each  -->

First, let's explore what this neural net would look like with only a single neuron. Even though it is only one neuron, it will need to have 27 weights, since all our inputs are size 27 and need a weight for each element within them.

In [8]:
# Make the weights contain random numbers from a normal distribution.
W = torch.randn((27, 1))    # Only one neuron so far, with one weight for each element in our input.
W

tensor([[-1.6458],
        [-0.8586],
        [-0.0919],
        [ 1.4253],
        [-1.0649],
        [-1.5436],
        [-3.0012],
        [-0.9322],
        [-1.7632],
        [ 1.1040],
        [-0.1884],
        [ 0.5235],
        [-0.2576],
        [-0.6203],
        [-0.9888],
        [-0.7899],
        [ 0.2243],
        [-0.1580],
        [-0.2241],
        [ 1.4914],
        [ 0.5052],
        [ 0.2037],
        [ 0.2657],
        [-0.4969],
        [-1.1664],
        [ 0.9730],
        [-0.8801]])

The first letter in X is ".", which is encoded as a 1 followed by 26 zeros. Thus, multiplying it with W would apply the dot product, selecting the first row of W (since all the other weights would get multiplied by 0).

In [9]:
# We expect the outcome here to be equal to W[0].
Xenc[0] @ W

tensor([-1.6458])

<!-- This only gives us a single value per input. We want  -->
<!-- This value can be used to calculate the probability  -->
A single neuron isn't enough here, since we need a neuron for each possible bigram pair so we can use it can calculate their probabilities. We can fix this by using a full layer of 27 neurons, one for each possible label.

<!-- Instead, let's make this layer contain a neuron for each possible label -->

In [39]:
# New weights should have 27 columns; one for each neuron.
W = torch.randn((27, 27))
# Let's take a peek at the first row of weights.
W[0]

tensor([ 0.0763,  1.7094, -0.0799,  0.8039,  0.9020, -0.2322,  0.5570,  1.0281,
        -1.0990, -0.0564, -0.6055,  0.2821,  0.1357, -0.4425, -1.2244, -1.1211,
         0.3559,  1.1136,  0.9501,  0.2514, -0.4165,  0.2189, -0.0048, -0.5803,
         0.0824, -1.3561,  0.5279])

<!-- Now multiplying it by Xenc should result in a vector of length 27, as it will -->
Now the code `Xenc[0] @ W` will instead select the first weight of each neuron and result in a vector of all those selected weights.

In [41]:
# This should result in the same values as shown above.
Xenc[0] @ W

tensor([ 0.0763,  1.7094, -0.0799,  0.8039,  0.9020, -0.2322,  0.5570,  1.0281,
        -1.0990, -0.0564, -0.6055,  0.2821,  0.1357, -0.4425, -1.2244, -1.1211,
         0.3559,  1.1136,  0.9501,  0.2514, -0.4165,  0.2189, -0.0048, -0.5803,
         0.0824, -1.3561,  0.5279])

Likewise, each other element of Xenc selects the row of W corresponding to the position of the 1 in its one-hot encoding. For example, the input letter "a" will select row 1, the input b will select row 2, and so on. Since each letter corresponds to a unique row in the matrix, we want that row to represent the probabilities of each next letter following the input letter that selected it.

Right now, however, our weights don't look very much like probabilities. They are negative and don't sum up to 1 (100%). We can fit the issue of negatives by exponentiating the values, making  all the digits positive while keeping values that were negative smaller than values that were positive. Then we can turn this into a probability by summing up each row and setting the weights for that row equal to their fraction of that sum. This process is called the softmax function.

<!-- Then if we sum up the rows, we can set each weight equal to its fraction of that sum,  -->

<!-- Then making them sum to 1 is a simple matter of setting them all equal to their  -->
<!-- To fix this, we will treat the output of our model as logits and apply the softmax function to it. This should result in each row having a sum of 1 (100%). -->

<!-- Thus, we'd like the rows to tell us the probability of each letter appearing after the letter that selects that row. -->
<!-- Thus, each row can be thought of as the  -->

In [46]:
logits = Xenc @ W
# After exponentiating, these values can be trained to find the count of each second letter in the bigram.
counts = logits.exp()
# Sum the column vectors to calculate the sum of each row, then divide each element in the row by the result to get their probabilities.
probs = counts / counts.sum(dim=1, keepdims=True)
probs

tensor([[0.0284, 0.1456, 0.0243,  ..., 0.0286, 0.0068, 0.0447],
        [0.0045, 0.0095, 0.0589,  ..., 0.0055, 0.1205, 0.0188],
        [0.0472, 0.0222, 0.0123,  ..., 0.0049, 0.0612, 0.0220],
        ...,
        [0.0104, 0.0089, 0.0651,  ..., 0.0099, 0.0077, 0.0136],
        [0.0146, 0.0598, 0.0418,  ..., 0.0349, 0.0080, 0.0565],
        [0.0196, 0.0242, 0.0164,  ..., 0.0034, 0.0034, 0.0069]])

Now that our model is set up, we're ready to train it.